In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from datetime import datetime
import numpy as np
from bs4 import BeautifulSoup
import requests

In [73]:
def get_info_from_track_name(sp, name):
    results = sp.search(q='track:' + name, type='track')
    items = results['tracks']['items']
    if len(items) > 0 :
        try : 
            release = datetime.strptime(items[0]['album']['release_date'], '%Y-%m-%d') 
        except ValueError : 
            try :
                release = datetime.strptime(items[0]['album']['release_date'], '%Y-%m')
            except ValueError : 
                release = datetime.strptime(items[0]['album']['release_date'], '%Y')
    return [str(name), str(items[0]['id']), release, str(len(items[0]['artists']))] if len(items) > 0 else None

In [169]:
def keep_col_in_feature(feature, to_keep):
    return {your_key : feature[your_key] if feature is not None else None for your_key in to_keep}

In [160]:
def get_spotify_info_from_names(list_tracks_names):
    ### init spotify api
    client_id = "f0f3c8adf2504e9fade50d0d2a174819"
    client_secret = "5c068fdd6f0e421e9385a2300d44bf84"
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    feature_cols = ["duration_ms", "key", "mode", "time_signature", "acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "valence", "tempo"]
    ### Get spoty names, ids, release, number of artists
    all_inf = np.array(list(map(lambda x : get_info_from_track_name(sp, x), list_tracks_names)))
    list_tracks_nanmes_spot, list_tracks_ids, list_tracks_release, list_tracks_nb_art = np.array(list(map(lambda x : x, all_inf[all_inf != np.array(None)]))).transpose()
    ### Get tracks's features from ids
    list_id = np.array_split(list_tracks_ids, len(list_tracks_ids)//100 + 1)
    features = []
    for i in list_id:
        features.append(sp.audio_features(i))
    tracks_features = [item for sub in features for item in sub]
    list_tracks_features = pd.DataFrame(list(map(lambda x : keep_col_in_feature(x, feature_cols), tracks_features)), index = list_tracks_nanmes_spot)
    list_tracks_features['release'] = list_tracks_release
    list_tracks_features['nb_art'] = list_tracks_nb_art
    return list_tracks_features

### Get track names to analyse

In [13]:
list_tracks_names = ['Despacito', 'Gangnam Style']

In [23]:
summer_songs = pd.read_csv('/home/p_paki/Documents/Kit DataScience/Hackaton/Summer_songs_pv.csv', delimiter=';', usecols=['Title', 'Artist', 'Year', '0'])

In [26]:
list_tracks_names = np.array(summer_songs['Title'])

In [173]:
result = get_spotify_info_from_names(list_tracks_names)

retrying ...1secs


In [174]:
result

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,release,nb_art
Doesn't Really Matter,0.000037,0.460,83058.0,0.771,0.449000,0.0,0.6400,-5.877,1.0,0.0362,150.993,4.0,0.9620,2017-04-20,1
Incomplete,0.282000,0.750,274227.0,0.446,0.000000,1.0,0.1190,-8.059,0.0,0.0759,119.489,4.0,0.2910,1999-01-01,1
"Jumpin', Jumpin'",0.005430,0.771,230200.0,0.685,0.001570,1.0,0.0537,-4.639,1.0,0.0567,88.997,4.0,0.6830,1999-07-26,1
Bent,0.213000,0.726,188387.0,0.711,0.000001,7.0,0.3260,-7.728,1.0,0.0414,105.034,4.0,0.7080,1979-01-01,1
No More,0.021700,0.766,168636.0,0.674,0.000059,0.0,0.2580,-6.245,1.0,0.0538,121.012,4.0,0.2140,2018-08-31,2
I Wanna Know,0.186000,0.548,272394.0,0.532,0.000263,5.0,0.2170,-7.596,1.0,0.0323,85.030,4.0,0.4050,2013-09-09,1
Kryptonite,0.006640,0.545,233933.0,0.865,0.000011,11.0,0.1680,-5.708,0.0,0.0286,99.010,4.0,0.5430,2000-01-01,1
Wifey,0.067000,0.829,243159.0,0.652,0.000000,7.0,0.0812,-8.693,0.0,0.1080,99.581,4.0,0.7260,2004-04-05,1
Absolutely (Story Of A Girl),0.000891,0.481,189333.0,0.940,0.000000,7.0,0.0939,-5.204,1.0,0.0660,96.493,4.0,0.6600,2000-05-16,1
It's Gonna Be Me,0.182000,0.416,202980.0,0.797,0.000000,10.0,0.1130,-3.874,0.0,0.0622,160.046,4.0,0.5990,2017-11-24,1


In [172]:
result.to_pickle("/home/p_paki/Documents/Kit DataScience/Hackaton/final.pkl")